## BREAST CANCER DETECTION

In [254]:
#importing pandas
import pandas as pd

df = pd.read_csv("data.csv")
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [255]:
df.isna().sum()

id                           0
diagnosis                    0
radius_mean                  0
texture_mean                 0
perimeter_mean               0
area_mean                    0
smoothness_mean              0
compactness_mean             0
concavity_mean               0
concave points_mean          0
symmetry_mean                0
fractal_dimension_mean       0
radius_se                    0
texture_se                   0
perimeter_se                 0
area_se                      0
smoothness_se                0
compactness_se               0
concavity_se                 0
concave points_se            0
symmetry_se                  0
fractal_dimension_se         0
radius_worst                 0
texture_worst                0
perimeter_worst              0
area_worst                   0
smoothness_worst             0
compactness_worst            0
concavity_worst              0
concave points_worst         0
symmetry_worst               0
fractal_dimension_worst      0
Unnamed:

In [256]:
#dropping "Unnmaed: 32" column
df.drop(["Unnamed: 32"], axis=1,inplace=True)

In [257]:
df["diagnosis"].value_counts()

diagnosis
B    357
M    212
Name: count, dtype: int64

### Using Random Forest

In [259]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score, confusion_matrix, accuracy_score

X = df.drop(["diagnosis"], axis = 1)
y = df["diagnosis"].map({"B": 0, "M":1})

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)
rf_clf = RandomForestClassifier(random_state = 42)
rf_clf.fit(X_train, y_train)
y_pred = rf_clf.predict(X_test)
print(f"Accuracy Score: {accuracy_score(y_test, y_pred)}")
print(f"Recall: {recall_score(y_test, y_pred, pos_label=1)}")
#print(f"c matrix: {confusion_matrix(y_test, y_pred)}")

Accuracy Score: 0.956140350877193
Recall: 0.8809523809523809


Here, **recall** is necessary to be higher as the model identifies most of the actual cancer cases, reducing the likelihood of false negatives where a patient with cancer is mistakenly classified as healthy. Missing a cancer diagnosis can delay critical treatment, making recall more important than other metrics like precision or accuracy.

In [261]:
#performing cross validation
from sklearn.model_selection import cross_val_score, KFold
import numpy as np

kf = KFold(n_splits = 10, shuffle=True, random_state=42)
cv_results = cross_val_score(rf_clf, X, y, cv = kf)
np.mean(cv_results)

0.9648496240601503

### Using KNN

In [263]:
from sklearn.neighbors import KNeighborsClassifier
knn_clf = KNeighborsClassifier(n_neighbors=3)
knn_clf.fit(X_train, y_train)
y_pred = knn_clf.predict(X_test)
print(f"Accuracy Score: {accuracy_score(y_pred, y_test)}")
print(f"Recall: {recall_score(y_test, y_pred, pos_label=1)}")

Accuracy Score: 0.7456140350877193
Recall: 0.5238095238095238


### Using XGBoost

In [265]:
import xgboost as xgb

xg_cl = xgb.XGBClassifier(objective="binary:logistic", n_estimators=200, seed=42)
xg_cl.fit(X_train, y_train)
y_pred = xg_cl.predict(X_test)
#confusion_matrix(y_pred, y_test)
print(f"Accuracy Score: {accuracy_score(y_pred, y_test)}")
print(f"Recall: {recall_score(y_pred, y_test)}")

Accuracy Score: 0.9736842105263158
Recall: 1.0
